## mount to google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/AICamp2023/MMSegmentation

Mounted at /content/drive
/content/drive/MyDrive/AICamp2023/MMSegmentation


## installaltion

In [2]:
!pip install -U openmim
!mim install mmengine
!mim install 'mmcv>=2.0.0rc3'
!mim install "mmdet>=3.0.0rc6"

#!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -v -e .
%cd ..

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 6.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 32.3 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.3.0
    Uninstalling platformdirs-3.3.0:
      Successfully uninstalled platformdirs-3.3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 24.6 MB/s eta 0:00:00
Looking in indexes: https://py

## GPU information

In [3]:
import os
import mmcv
import mmengine
import torch

if torch.cuda.is_available():
    print('CUDA is available')
    device_count = torch.cuda.device_count()
    print(f'Total number of CUDA devices: {device_count}')
    for i in range(device_count):
        device_name = torch.cuda.get_device_name(i)
        print(f'Device {i}: {device_name}')
else:
    print('CUDA is not available')

CUDA is available
Total number of CUDA devices: 1
Device 0: NVIDIA A100-SXM4-40GB


## fetch configs

In [8]:
!mim search mmsegmentation --model pspnet

--------------------------------------------------------------------------------
config id: mobilenet-v2-d8_pspnet_4xb2-80k_cityscapes-512x1024
architecture                         m-v2-d8,pspnet
batch_size                           8
cityscapes/miou                      70.23
config                               configs/mobilenet_v2/mobilenet-v2-d8_psp...
memory(GB)                           3.6
model                                pspnet
paper                                Title,URL
readme                               configs/pspnet/README.md
training_data                        cityscapes
training_resources                   4x v100 gpus
weight                               https://download.openmmlab.com/mmsegment...
--------------------------------------------------------------------------------
config id: mobilenet-v2-d8_pspnet_4xb4-160k_ade20k-512x512
ade20k/miou                          29.68
architecture                         m-v2-d8,pspnet
batch_size                       

In [9]:
!mim download mmsegmentation --config pspnet_r101-d8_4xb4-80k_ade20k-512x512 --dest Configs

processing pspnet_r101-d8_4xb4-80k_ade20k-512x512...
pspnet_r101-d8_512x512_80k_ade20k_20200614_031423-b6e782f0.pth exists in /content/drive/MyDrive/AICamp2023/MMSegmentation/Configs
Successfully dumped pspnet_r101-d8_4xb4-80k_ade20k-512x512.py to /content/drive/MyDrive/AICamp2023/MMSegmentation/Configs


## prepare WatermelonDataset

In [4]:
import os
import shutil
import cv2
from tqdm import tqdm

data_dir = 'Watermelon87_Semantic_Seg_Mask'

for folder_name in tqdm(['img_dir//train', 'img_dir//val'], total=2):
    folder_path = os.path.join(data_dir, folder_name)
    file_names = os.listdir(folder_path)
    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        image = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
        base_name, extension = os.path.splitext(file_path)
        new_filepath = base_name + '.jpg'
        cv2.imwrite(new_filepath, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        os.remove(file_path)

train_names = os.listdir(os.path.join(data_dir, 'img_dir//train'))
val_names = os.listdir(os.path.join(data_dir, 'img_dir//val'))

images_dir = os.path.join(data_dir, 'images'); os.makedirs(images_dir, exist_ok=True) # images dir
labels_dir = os.path.join(data_dir, 'labels'); os.makedirs(labels_dir, exist_ok=True) # masks dir
splits_dir = os.path.join(data_dir, 'splits'); os.makedirs(splits_dir, exist_ok=True) # train/val file dir

for folder_name in tqdm(['img_dir//train', 'img_dir//val', 'ann_dir//train', 'ann_dir//val'], total=4):
    source = os.path.join(data_dir, folder_name) # copy source
    if 'img_dir' in folder_name:
        destination = images_dir # destination
    elif 'ann_dir' in folder_name:
        destination = labels_dir # destination
    file_names = [name for name in os.listdir(source) if any(name.endswith(file_type) for file_type in ['jpg', 'png'])]
    for name in file_names:
        shutil.copy2(os.path.join(source, name), os.path.join(destination, name)) # copy files to the common folder

with open(os.path.join(splits_dir, 'train.txt'), 'w') as f:
    f.writelines(os.path.splitext(name)[0] + '\n' for name in train_names) # creat a txt file for train
with open(os.path.join(splits_dir, 'val.txt'), 'w') as f:
    f.writelines(os.path.splitext(name)[0] + '\n' for name in val_names) # create a txt file for validation

100%|██████████| 4/4 [00:49<00:00, 12.40s/it]


## register WatermelonDataset

In [5]:
import os
import shutil
import cv2
from tqdm import tqdm

data_dir = 'Watermelon87_Semantic_Seg_Mask'

list2copy = [
    'Configs//mmseg_watermelon.py',
    'Configs//pspnet_r101-d8_4xb4-80k_ade20k-512x512.py',
    'Configs//pspnet_r101-d8_512x512_80k_ade20k_20200614_031423-b6e782f0.pth'
]

for dir in tqdm(list2copy, total=len(list2copy)):
    shutil.copy2(dir, 'mmsegmentation')

list2copy2 = [
    'Configs//__init__.py',
    'Configs//watermelonDataset.py'
]

for dir in tqdm(list2copy2, total=len(list2copy2)):
    shutil.copy2(dir, 'mmsegmentation//mmseg//datasets')

shutil.copytree(data_dir, 'mmsegmentation//Watermelon87_Semantic_Seg_Mask')

100%|██████████| 2/2 [00:01<00:00,  1.78it/s]


'mmsegmentation//Watermelon87_Semantic_Seg_Mask'

## training

In [6]:
%cd mmsegmentation
!mim train mmseg mmseg_watermelon.py

Streaming output truncated to the last 5000 lines.
06/15 20:41:48 - mmengine - INFO - Epoch(train) [2631][12/12]  lr: 1.2500e-06  eta: 0:08:40  time: 0.1116  data_time: 0.0036  memory: 4721  loss: 0.0377  decode.loss_ce: 0.0247  decode.acc_seg: 99.3217  aux.loss_ce: 0.0129  aux.acc_seg: 99.2060
06/15 20:41:48 - mmengine - INFO - per class results:
06/15 20:41:48 - mmengine - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 98.63 |  99.3 |
|    red     | 97.81 |  98.9 |
|   green    | 94.22 | 96.88 |
|   white    | 93.07 |  96.7 |
| seed-black | 76.56 | 85.84 |
| seed-white | 75.29 | 82.09 |
+------------+-------+-------+
06/15 20:41:48 - mmengine - INFO - Epoch(val) [2631][24/24]    aAcc: 98.7100  mIoU: 89.2600  mAcc: 93.2800  data_time: 0.0028  time: 0.0243
06/15 20:41:50 - mmengine - INFO - Exp name: mmseg_watermelon_20230615_190937
06/15 20:41:50 - mmengine - INFO - Epoch(train) [2632][12/12]  lr: 1.2500e-06  eta: 0:

## testing

In [9]:
!mim test mmseg ..//Models//PSPNet//mmseg_watermelon.py --checkpoint \
        ..//Models//PSPNet//best_mIoU_epoch_1963.pth --gpus 1

Testing command is /usr/bin/python3 /content/drive/MyDrive/AICamp2023/MMSegmentation/mmsegmentation/mmseg/.mim/tools/test.py ..//Models//PSPNet//mmseg_watermelon.py ..//Models//PSPNet//best_mIoU_epoch_1963.pth --launcher none. 
06/15 20:54:35 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 585173119
    GPU 0: NVIDIA A100-SXM4-40GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP

## inference

In [10]:
%cd ..

/content/drive/MyDrive/AICamp2023/MMSegmentation


In [11]:
from mmseg.utils import register_all_modules
from mmseg.apis import init_model, inference_model, show_result_pyplot

register_all_modules(init_default_scope=False)
model = init_model('Models//PSPNet//mmseg_watermelon.py', 'Models//PSPNet//best_mIoU_epoch_1963.pth', device='cuda')
result = inference_model(model, 'Test//IMG_9858.jpg')
vis_iamge = show_result_pyplot(model, 'Test//IMG_9858.jpg', result, out_file='Test//IMG_9858_result.jpg')

/content/drive/MyDrive/AICamp2023/MMSegmentation/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/content/drive/MyDrive/AICamp2023/MMSegmentation/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: Models//PSPNet//best_mIoU_epoch_1963.pth
06/15 21:01:52 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
